In [53]:
import os

import numpy as np
from scipy.sparse import load_npz
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import pandas as pd

import torch
import torch_geometric
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import GATConv
from torch import sparse

import matplotlib.pyplot as plt
from torch_sparse import SparseTensor, matmul

In [129]:
adj_t = SparseTensor(
    row=torch.tensor([0, 0, 1, 0, 2, 3, 2]),  # Source nodes
    col=torch.tensor([1, 2, 2, 3, 1, 2, 0]),  # Target nodes
    value=torch.tensor([0.5, 0.8, 0.6, 0.1, 0.2, 0.9, 0.3]),  # Edge weights
    sparse_sizes=(4, 4)  # 3 nodes in total
)

In [130]:
adj_t.to_dense()

tensor([[0.0000, 0.5000, 0.8000, 0.1000],
        [0.0000, 0.0000, 0.6000, 0.0000],
        [0.3000, 0.2000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9000, 0.0000]])

In [131]:
edge_weights = torch.nn.Parameter(torch.rand(adj_t.nnz(), dtype=torch.float32))
#edge_weights = torch.tensor([1, 2, 3, 4], dtype=torch.float32)

In [132]:
coo = adj_t.coo()
row_indices = coo[0]
col_indices = coo[1]

row_sum = torch.zeros(4, dtype=torch.float32)
row_sum = row_sum.scatter_add(0, row_indices, edge_weights)
row_sum = row_sum + 1e-8
row_sum

tensor([1.0749, 0.9080, 1.2034, 0.4641], grad_fn=<AddBackward0>)

In [133]:
normalized_weights = edge_weights / row_sum[row_indices]

In [134]:
adj_t_updated = adj_t.set_value(normalized_weights)

In [135]:
adj_t_updated.to_dense()

tensor([[0.0000, 0.1737, 0.5609, 0.2655],
        [0.0000, 0.0000, 1.0000, 0.0000],
        [0.4867, 0.5133, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000]], grad_fn=<IndexPutBackward0>)